# Caracterización de distribuciones unidimensionales

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

In [ ]:
data = np.loadtxt("dataset.txt")
plt.hist(data, bins=50);

# Medidas de Centrales

### Media

Si tenemos un conjunto de puntos N puntos denotados $x_i$, la media se define como

$$ \frac{1}{N} \sum_{i=1}^N x_i $$

Una forma de calcular manualmente la media viene dada por:

In [ ]:
def funcion_media(xs):
    sumatoria = 0
    for x in xs:
        sumatoria += x
    return sumatoria / len(xs)
print(funcion_media([3, 5, 2, 6]))

Pero en esta era moderna no deberíamos tener que escribir la función nosotros mismos. Podemos usar `np.mean`. Si queremos que los puntos de datos tengan diferentes pesos, podemos usar `np.average` en su lugar. (Por ejemplo, el lanzamiento de dados solo puede registrar el valor y la cantidad de veces, no cada lanzamiento individual).

In [ ]:
mean = np.mean(data)
print(mean, data.mean(), np.average(data))

### Mediana

Ordenar todos los datos y escoger el elemento del medio. Esa es la mediana. `[1,3,5,7,7]` tiene una mediana de `5`. Así es como se puede encontrar manualmente la mediana:

In [ ]:
def funcion_mediana(xs):
    mediana = len(xs) // 2
    if len(xs) % 2 == 1:
        return sorted(xs)[mediana]
    else:
        return 0.5 * np.sum(sorted(xs)[mediana - 1:mediana + 1])
print(funcion_mediana([7, 7, 3, 1, 4, 5]))

En el caso que hubiera dos valores medianos, se hace la media aritmética entre ambios valores.

No uses la función, usa `np.median`

In [ ]:
median = np.median(data)
print(median)

In [ ]:
outlier = np.insert(data, 0, 5000)
plt.hist(data, label="Data", bins=50);
plt.axvline(np.mean(data), ls="--", label="Media Datos")
plt.axvline(np.median(data), ls=":", label="Mediana Datos")
plt.axvline(np.mean(outlier), c='r', ls="--", label="Media con Outlier", alpha=0.7)
plt.axvline(np.median(outlier), c='r', ls=":", label="Mediana con Outlier", alpha=0.7)
plt.legend()
plt.xlim(0,20);

### Mode

In [ ]:
def obtener_moda(xs):
    valores, recuento = np.unique(xs, return_counts=True)
    max_recuento_indice = np.argmax(recuento)
    return valores[max_recuento_indice]
print(obtener_moda([1,7,2,5,3,3,8,3,2]))

In [ ]:
moda = st.mode(data)
print(moda)

In [ ]:
hist, bordes = np.histogram(data, bins=100)
bordes_centers = 0.5 * (bordes[1:] + bordes[:-1])
moda = bordes_centers[hist.argmax()]
print(moda)

In [ ]:
kde = st.gaussian_kde(data)
xvals = np.linspace(data.min(), data.max(), 1000)
yvals = kde(xvals)
mode = xvals[yvals.argmax()]
plt.hist(data, bins=1000, density=True, label="Datos hist", histtype="step")
plt.plot(xvals, yvals, label="KDE")
plt.axvline(mode, label="Moda")
plt.legend();

# Comparación

In [ ]:
plt.hist(data, bins=100, label="Datos", alpha=0.5)
plt.axvline(mean, label="Media", ls="--", c='#f9ee4a')
plt.axvline(median, label="Mediana", ls="-", c='#44d9ff')
plt.axvline(mode, label="Moda", ls=":", c='#f95b4a')
plt.legend();

# Medidas de dispersión y forma

* Varianza
* Desviación Estándar
* asimetría
* curtosis

### Varianza

La varianza de una distribución es una medida de cuánto se dispersa alrededor de la media. Un toque más formal, es el valor esperado de la desviación al cuadrado de la media. Aún más formalmente, está dada por

$$ Var = \frac{1}{N} \sum_{i=1}^N (x_i - \mu)^2, $$

donde $\mu$ es la media del conjunto de datos $x$, como se describe en la sección anterior. Tenga en cuenta que hay un punto fino sobre si debe dividir por $N$ o $N-1$. 
**Eso lo resolveremos cuando debemos inferencia**

Aquí hay una forma manual de calcularlo:

In [ ]:
def obtener_varianza(xs):
    media = np.mean(xs)
    suma = 0
    for x in xs:
        suma += (x - media)**2
    return suma / (len(xs))
print(obtener_varianza([1,2,3,4,5]))

In [ ]:
varianza = np.var(data, ddof=1)
print(varianza)

### Desviación Estandar

Este es sencillo. Es la raíz cuadrada de la varianza. Así que es el valor absoluto esperado de la desviación de la media. Y podemos usar `np.std` para ello o `pd.DataFrame.std` ([documentación](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.std.html))

In [ ]:
std = np.std(data)
print(std, std**2)

### Poder combianado

In [ ]:
xs = np.linspace(data.min(), data.max(), 100)
ys = st.norm.pdf(xs, loc=mean, scale=std)

plt.hist(data, bins=50, density=True, histtype="step", label="Datos")
plt.plot(xs, ys, label="Aproximación Normal ")
plt.legend()
plt.ylabel("Probabilidad");

### asimetría

En esta sección podría dejar caer la palabra "momento" unas cuantas veces. Hay algunas formas estandarizadas de cuantificar los "momentos". El primer momento es cero por definición. La segunda es la varianza. El tercero es la asimetría, que a menudo se define como $\gamma_1$

$$ \gamma_1 = \frac{\kappa_3}{\kappa_2^{3/2}} = \frac{E[(x-\mu)^3]}{E[(x-\mu)^2]^{3/2}} $$

In [ ]:
def obtener_asimetria(xs):
    media = np.mean(xs)
    var = np.var(xs)
    suma = 0
    for x in xs:
        suma += (x - media)**3
    return (suma / (len(xs))) / (var ** 1.5)
print(obtener_asimetria([1,2,3,4,5]))

In [ ]:
skewness = st.skew(data)
print(skewness, obtener_asimetria(data))

In [ ]:
xs = np.linspace(data.min(), data.max(), 100)
ys1 = st.norm.pdf(xs, loc=mean, scale=std)
ys2 = st.skewnorm.pdf(xs, skewness, loc=mean, scale=std)

plt.hist(data, bins=50, density=True, histtype="step", label="Datos")
plt.plot(xs, ys1, label="Aproximación Normal")
plt.plot(xs, ys2, label="Aproximación Asimetria Normal")
plt.legend()
plt.ylabel("Probabilidad");

In [ ]:
xs = np.linspace(data.min(), data.max(), 100)
ys1 = st.norm.pdf(xs, loc=mean, scale=std)
ps = st.skewnorm.fit(data)
ys2 = st.skewnorm.pdf(xs, *ps)

plt.hist(data, bins=50, density=True, histtype="step", label="Datos")
plt.plot(xs, ys1, label="Aproximación Normal")
plt.plot(xs, ys2, label="Aproximación Asimetria Normal")
plt.legend()
plt.ylabel("Probabilidad");

### Curtosis

El siguiente momento, y el último que consideraremos, es la curtosis. Tiene una definición similar y, a menudo, se representa como $\kappa$ o $\gamma_2$:

$$ \kappa = \frac{E[(x-\mu)^4]}{E[(x-\mu)^2]^{4/2}} $$

In [ ]:
def obtener_curtosis(xs):
    media = np.mean(xs)
    var = np.var(xs)
    suma = 0
    for x in xs:
        suma += (x - media)**4
    return (suma / (len(xs))) / (var ** 2)
print(obtener_curtosis([1,2,3,4,5]))

In [ ]:
curtosis = st.kurtosis(data, fisher=False)
print(curtosis, obtener_curtosis(data))

### Percentiles

In [ ]:
ps = np.linspace(0, 100, 10)
x_p = np.percentile(data, ps)

xs = np.sort(data)
ys = np.linspace(0, 1, len(data))

plt.plot(xs, ys * 100, label="ECDF")
plt.plot(x_p, ps, label="Percentiles", marker=".", ms=10)
plt.legend()
plt.ylabel("Percentil");

In [ ]:
ps = 100 * st.norm.cdf(np.linspace(-4, 4, 30))
x_p = np.percentile(data, ps)

xs = np.sort(data)
ys = np.linspace(0, 1, len(data))

plt.plot(xs, ys * 100, label="ECDF")
plt.plot(x_p, ps, label="Percentiles", marker=".", ms=10)
plt.legend()
plt.ylabel("Percentil");

In [ ]:
ps = 100 * st.norm.cdf(np.linspace(-3, 3, 50))
ps = np.concatenate(([0], ps, [100]))  # Hay un error en la forma de inserción de hacerlo, esto es mejor
x_p = np.percentile(data, ps)

xs = np.sort(data)
ys = np.linspace(0, 1, len(data))

plt.plot(xs, ys * 100, label="ECDF")
plt.plot(x_p, ps, label="Percentiles", marker=".", ms=10)
plt.legend()
plt.ylabel("Percentile");

In [ ]:
from scipy.interpolate import interp1d

n = int(1e6)
u = np.random.uniform(size=n)
muestra_percentile_1 = interp1d(ps / 100, x_p)(u)

_, bins, _ = plt.hist(data, bins=50, density=True, alpha=0.3, label="Datos")
plt.hist(muestra_percentile_1, bins=bins, density=True, histtype="step", label="Percentiles")
plt.ylabel("Probability")
plt.legend();